In [ ]:
import pycinema
import PIL
import numpy as np
from matplotlib import pyplot

params = [61.33855781651522, 334.0391481158407, 13.0]
# params = [61.33855781651522, 334.0391481158407, 14.0]
# params = [-10, 5.0, 3]
size = 6

# Open Cinema Database
cdb  = pycinema.CinemaDatabaseReader();
cdb.inputs.Path.set( 'testing/gold/artifact/ML_ImageGeneratorTimevarying_Test.cdb' );

# Select Some Data Products
query = pycinema.DatabaseQuery();
query.inputs.Table.set(cdb.outputs.Table);
qstring = "SELECT * FROM INPUT WHERE phi = '{}' AND theta = '{}' AND time = '{}'".format(params[0], params[1], params[2])
query.inputs.Query.set(qstring);

# Read Data Products
imageReader = pycinema.ImageReader();
imageReader.inputs.Table.set(query.outputs.Table);

# Add annotation
annotation = pycinema.Annotation()
annotation.inputs.Images.set(imageReader.outputs.Images,False)
annotation.inputs.Color.set((255,0,0))
annotation.inputs.Ignore.set(['FILE', 'id', 'phi', 'theta'])
annotation.inputs.Size.set(size)
annotation.inputs.XY.set((5,5))

# Display Results
images = annotation.outputs.Images.get();
if len(images) > 0:
    for i in range(len(images)):   
        image = images[i]
        pyplot.imshow(image.channel['RGBA'])
        pyplot.axis('off')
        pyplot.show()

else:
    ## Create the instance of the filter
    mlfilter = pycinema.ImageGeneratorCNN()

    ## Input trained model file
    mlfilter.inputs.Model.set('testing/gold/artifact/MLModels/Model_MLImageGenerator_Isabel_timevarying_2000.pth',False)
    mlfilter.inputs.Device.set('cpu',False)
    mlfilter.inputs.Params.set([params],False)
    mlfilter.inputs.VP.set(3,False)
    mlfilter.inputs.VPO.set(256,False)
    mlfilter.inputs.Channel.set(8,False)

    # Add annotation
    annotation.inputs.Images.set(mlfilter.outputs.Images,False)

    # update the pipeline
    mlfilter.update()

    # show the generated images
    images = annotation.outputs.Images.get();
    for i in range(len(images)):
        image = images[i]
        pyplot.imshow(image.channel['RGBA'])
        pyplot.axis('off')
        pyplot.show()